# Syllabi Analysis and NLP for potential course selection

This notebook constructs a dataframe containing information about potential courses of interest for an RCT. It begins by extracting text from a large database of syllabi and identifying relevant characteristics using regular expressions. Then, it extracts additional course data from a complex tabulated PDF and enriches the main dataframe by matching course codes.

The resulting dataframe was instrumental in selecting the course for the randomized control trial, as it facilitated the visualization of crucial course characteristics, such as potential observations, references to group projects in the syllabus, faculty details, and more.

## Setup

In [1]:
import os
import fitz  # PyMuPDF
import random
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import re
from IPython.display import display, HTML
from collections import defaultdict
import numpy as np

In [2]:
os.getcwd()

'c:\\Users\\Matias Villalba\\Documents\\GitHub\\VRI_UP_Networks\\code'

In [3]:
os.chdir("../")
root = os.getcwd()
data = root + r'\data'

## Syllabi text Analysis

### Text Extraction

We define functions to programatically extract the text of all of our syllabus pdf files. 

In [4]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file.

    Parameters:
    pdf_path (str): The path to the PDF file.

    Returns:
    str: The extracted text from the PDF.
    """
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    
    # Initialize an empty string to hold the text
    text = ""

    # Iterate through each page
    for page_num in range(pdf_document.page_count):
        # Get the page
        page = pdf_document[page_num]
        # Extract text from the page
        text += page.get_text()

    # Close the PDF file
    pdf_document.close()

    return text

In [5]:
extract_text_from_pdf(data + r'\syllabus\1_Administración\(1) 142085 - Gestión de la Sostenibilidad Social y Ambiental en las Empresas (2023-I).pdf')

'Sílabo\n142085 - Gestión de la Sostenibilidad Social y Ambiental en las Empresas\nI.\nInformación general\nNombre del Curso: Gestión de la Sostenibilidad Social y Ambiental en las Empresas\nCódigo del curso: 142085\nDepartamento Académico: Administración\nCréditos: 4\nHoras Teoría: 4\nHoras Práctica: 0\nPeriodo Académico: 2023-01-PRE\nSección: A\nModalidad: Presencial\nIdioma: Español\nDocente: SILVANA CARO VARGAS\nEmail docente: s.carov@up.edu.pe\nII.\nIntroducción\nEl curso promueve en los alumnos una actitud proactiva hacia la sostenibilidad social y ambiental de las\nempresas, que ha de evidenciarse en el ejercicio de los diversos roles que asumirán a lo largo de su\ntrayectoria profesional. A través del desarrollo del curso los alumnos conocerán, desde una perspectiva\nestratégica,  los  procesos  e  instrumentos  relacionados  con  la  gestión  sostenible  y  socialmente\nresponsable. Además, se espera aportar a la formación de un profesional técnicamente competente y\nsocialmen

In [6]:
pdfs = data + r'\syllabus'

In [7]:
filenames = []

for base, dirs, files in os.walk(pdfs):
    for filename in files:
        if (filename.lower().endswith('.pdf')):
            filenames.append(os.path.join(base, filename))

random.choices(filenames, k=5)

['c:\\Users\\Matias Villalba\\Documents\\GitHub\\VRI_UP_Networks\\data\\syllabus\\6_Finanzas\\1F0069 - Microeconomía Financiera (2023-I).pdf',
 'c:\\Users\\Matias Villalba\\Documents\\GitHub\\VRI_UP_Networks\\data\\syllabus\\4_Derecho\\186045 - Arbitraje (2023-I).pdf',
 'c:\\Users\\Matias Villalba\\Documents\\GitHub\\VRI_UP_Networks\\data\\syllabus\\1_Administración\\(2) 142085 - Gestión de la Sostenibilidad Social y Ambiental en las Empresas (2023-I).pdf',
 'c:\\Users\\Matias Villalba\\Documents\\GitHub\\VRI_UP_Networks\\data\\syllabus\\5_Economía\\130639 - Diseño y Evaluación Social de Proyectos (2023-I).pdf',
 'c:\\Users\\Matias Villalba\\Documents\\GitHub\\VRI_UP_Networks\\data\\syllabus\\8_Ingeniería\\170355 - Informática para los Negocios (2023-I).pdf']

In [8]:
len(filenames)

276

In [9]:
def extract_texts_from_pdfs(filepaths):
    """
    Extracts text from multiple PDF files using parallel processing.

    Parameters:
    filepaths (list): A list of file paths to the PDF files.

    Returns:
    pd.DataFrame: A DataFrame with the filenames and extracted texts.
    """
    # Use ProcessPoolExecutor for parallel processing
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(extract_text_from_pdf, filepaths))

    filenames = [os.path.basename(filepath) for filepath in filepaths]

    # Create a DataFrame from the results and filenames
    df = pd.DataFrame({'filepath': filepaths, 'filename': filenames, 'text': results})

    return df

In [10]:
df = extract_texts_from_pdfs(filenames)
df.head(10)

,filepath,filename,text
0,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(1) 142085 - Gestión de la Sostenibilidad Soc...,Sílabo\n142085 - Gestión de la Sostenibilidad ...
1,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(1) 144741 - Gestión de Personas (2023-I).pdf,Sílabo\n144741 - Gestión de Personas\nI.\nInfo...
2,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(2) 142085 - Gestión de la Sostenibilidad Soc...,\n1/12 \n \n \n \nSÍLABO \n \nI. \nInformació...
3,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(2) 144741 - Gestión de Personas (2023-I).pdf,\n \n \n1 \n \nSÍLABO \nI. Información g...
4,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,140663 - Negocios Inclusivos (2023-I).pdf,Sílabo\n140663 - Negocios Inclusivos\nI.\nInfo...
5,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,141036 - Análisis Multivariado para los Negoc...,Sílabo\n141036 - Análisis Multivariado para lo...
6,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,141038 - Fundamentos de las Ciencias Empresari...,Sílabo\n141038 - Fundamentos de las Ciencias E...
7,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,141041 - Gestión de la Cadena de Suministros ...,Sílabo\n141041 - Gestión de la Cadena de Sumin...
8,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,141042 - Investigación Aplicada a los Negocio...,\n5/5 \n \n \n \nSÍLABO \n \nI. \nInformación...
9,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,141401- Manejo Conflictos (2023 - I).pdf,\n1/2 \n \n \n \nSÍLABO \n \nI. \nInformación...


### Key characteristic identification

In [11]:
# Define regex pattern to identify references to group projects
project_pattern = re.compile(r'\b(proyecto(|s)\s+grupal(|es)|proyecto(|s)\s+en\s+grupo|trabajo(|s)\s+grupal(|es)|proyecto\s+final|group\s+project(|s)|trabajo(|s)\s+en\s+grupo|grupo(|s)\s+de\s+trabajo|caso(|s)\s+grupal(|es))\b', re.IGNORECASE)

# Create a new column to indicate references to group projects
df['group_project'] = df['text'].apply(lambda x: bool(project_pattern.search(x)))

In [12]:
# Define regex pattern to identify references to teamwork
team_pattern = re.compile(r'\b(trabajo\s+en\s+equipo|teamwork)\b', re.IGNORECASE)

# Create a new column to indicate references to teamwork
df['teamwork'] = df['text'].apply(lambda x: bool(team_pattern.search(x)))

In [13]:
# Function to extract course code
def extract_course_code(filename):
    match = re.search(r'(\b\d{6}|1MN\d{3}|1F\d{4}\b)', filename)
    if match:
        return match.group(1)
    return None

# Apply the function to the 'filename' column and create a new column 'course_code'
df['course_code'] = df['filename'].apply(extract_course_code)

In [14]:
# Faculty mapping
faculty_mapping = {
    '12': 'Humanidades',
    '13': 'Economía',
    '14': 'Administración',
    '15': 'Ciencia Sociales',
    '16': 'Contabilidad',
    '17': 'Ingeniería',
    '18': 'Derecho',
    '1F': 'Finanzas',
    '1MN': 'Marketing y Negocios'
}


# Function to determine faculty based on course code
def get_faculty(course_code):
    if course_code:
        prefix = course_code[:2] if course_code[:2] in faculty_mapping else course_code[:3]
        return faculty_mapping.get(prefix, 'Unknown')
    return 'Unknown'

df['faculty'] = df['course_code'].apply(get_faculty)

In [15]:
df.head(10)

,filepath,filename,text,group_project,teamwork,course_code,faculty
0,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(1) 142085 - Gestión de la Sostenibilidad Soc...,Sílabo\n142085 - Gestión de la Sostenibilidad ...,False,True,142085,Administración
1,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(1) 144741 - Gestión de Personas (2023-I).pdf,Sílabo\n144741 - Gestión de Personas\nI.\nInfo...,False,False,144741,Administración
2,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(2) 142085 - Gestión de la Sostenibilidad Soc...,\n1/12 \n \n \n \nSÍLABO \n \nI. \nInformació...,False,True,142085,Administración
3,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(2) 144741 - Gestión de Personas (2023-I).pdf,\n \n \n1 \n \nSÍLABO \nI. Información g...,False,False,144741,Administración
4,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,140663 - Negocios Inclusivos (2023-I).pdf,Sílabo\n140663 - Negocios Inclusivos\nI.\nInfo...,False,False,140663,Administración
5,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,141036 - Análisis Multivariado para los Negoc...,Sílabo\n141036 - Análisis Multivariado para lo...,True,False,141036,Administración
6,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,141038 - Fundamentos de las Ciencias Empresari...,Sílabo\n141038 - Fundamentos de las Ciencias E...,True,False,141038,Administración
7,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,141041 - Gestión de la Cadena de Suministros ...,Sílabo\n141041 - Gestión de la Cadena de Sumin...,False,False,141041,Administración
8,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,141042 - Investigación Aplicada a los Negocio...,\n5/5 \n \n \n \nSÍLABO \n \nI. \nInformación...,False,False,141042,Administración
9,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,141401- Manejo Conflictos (2023 - I).pdf,\n1/2 \n \n \n \nSÍLABO \n \nI. \nInformación...,False,False,141401,Administración


## Adding study plan information

We use this table that contains the mandatory freshman courses for each major to create dummies 

In [16]:
study_plan_df = pd.read_excel(data + r'\freshmen_courses.xlsx')
study_plan_df.head(10)

,mat_carrera,mat_plan_studio,cur_nombre_del_curso,course_code
0,Administración,2022-I PER,Fundamentos de las Ciencias Empresariales,141038
1,Administración,2022-I PER,Lenguaje I,120001
2,Administración,2022-I PER,Matemáticas I,138649
3,Administración,2022-I PER,Economía General I,132641
4,Contabilidad,2022-I PER,Fundamentos de las Ciencias Empresariales,141038
5,Contabilidad,2022-I PER,Lenguaje I,120001
6,Contabilidad,2022-I PER,Matemáticas I,138649
7,Contabilidad,2022-I PER,Economía General I,132641
8,Marketing,2022-I PER,Lenguaje I,120001
9,Marketing,2022-I PER,Matemáticas I,138649


In [17]:
study_plan_df['value'] = 1  # Create a column to use as values in the pivot table
study_plan_df['mat_carrera'] = "frshmn_" + study_plan_df['mat_carrera'].str.lower().str.replace('[ ,]+', '_', regex=True)
study_plan_df = study_plan_df.pivot_table(index=['course_code','cur_nombre_del_curso'], columns='mat_carrera', values='value', fill_value=0).reset_index()
study_plan_df['frshmn_any'] = 1
study_plan_df['course_code'] = study_plan_df['course_code'].astype(str)
study_plan_df.head()

mat_carrera,course_code,cur_nombre_del_curso,frshmn_administración,frshmn_contabilidad,frshmn_derecho,frshmn_economía,frshmn_finanzas,frshmn_humanidades_digitales,frshmn_ingeniería_de_la_información,frshmn_ingeniería_empresarial,frshmn_ingeniería_en_innovacion_y_diseño,frshmn_marketing,frshmn_negocios_internacionales,frshmn_politica_filosofia_y_economia,frshmn_any
0,120001,Lenguaje I,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
1,120252,Lectura crítica de la prensa digital y en las ...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,120260,"Introducción a la Política, Filosofía y Economía",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
3,132641,Economía General I,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
4,138649,Matemáticas I,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1


In [18]:
df = pd.merge(df, study_plan_df, on='course_code', how='left').drop(columns=['cur_nombre_del_curso'])
frshmn_columns = [col for col in df.columns if col.startswith('frshmn_')]
df[frshmn_columns] = df[frshmn_columns].fillna(0)
df.head()

,filepath,filename,text,group_project,teamwork,course_code,faculty,frshmn_administración,frshmn_contabilidad,frshmn_derecho,frshmn_economía,frshmn_finanzas,frshmn_humanidades_digitales,frshmn_ingeniería_de_la_información,frshmn_ingeniería_empresarial,frshmn_ingeniería_en_innovacion_y_diseño,frshmn_marketing,frshmn_negocios_internacionales,frshmn_politica_filosofia_y_economia,frshmn_any
0,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(1) 142085 - Gestión de la Sostenibilidad Soc...,Sílabo\n142085 - Gestión de la Sostenibilidad ...,False,True,142085,Administración,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(1) 144741 - Gestión de Personas (2023-I).pdf,Sílabo\n144741 - Gestión de Personas\nI.\nInfo...,False,False,144741,Administración,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(2) 142085 - Gestión de la Sostenibilidad Soc...,\n1/12 \n \n \n \nSÍLABO \n \nI. \nInformació...,False,True,142085,Administración,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(2) 144741 - Gestión de Personas (2023-I).pdf,\n \n \n1 \n \nSÍLABO \nI. Información g...,False,False,144741,Administración,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,140663 - Negocios Inclusivos (2023-I).pdf,Sílabo\n140663 - Negocios Inclusivos\nI.\nInfo...,False,False,140663,Administración,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df[df['course_code']=='141038']

,filepath,filename,text,group_project,teamwork,course_code,faculty,frshmn_administración,frshmn_contabilidad,frshmn_derecho,frshmn_economía,frshmn_finanzas,frshmn_humanidades_digitales,frshmn_ingeniería_de_la_información,frshmn_ingeniería_empresarial,frshmn_ingeniería_en_innovacion_y_diseño,frshmn_marketing,frshmn_negocios_internacionales,frshmn_politica_filosofia_y_economia,frshmn_any
6,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,141038 - Fundamentos de las Ciencias Empresari...,Sílabo\n141038 - Fundamentos de las Ciencias E...,True,False,141038,Administración,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


## Tabulated PDF course data extraction

This part of the notebook extracts structured data from a complexly formatted PDF file containing course information. 

Said doocument is available <a href="https://mvillalbao.github.io/assets/pdf/VRI_UP_Networks_data/data/Oferta-Academica-2024-II-V5.pdf" target="blank">here</a>

In [20]:
text = extract_text_from_pdf(data+ r'\Oferta-Academica-2024-II-V5.pdf')

In [21]:
text.splitlines()[:30]

['Secc',
 'Tipo',
 'Docentes',
 'Cred',
 'Teoría o Práctica',
 'Día',
 'Horario',
 'Duración',
 'Cupo',
 's',
 'Aula',
 'CURSOS ACADÉMICOS',
 '120266 - Antiguo Perú, Arqueología, Museos y Nuevas Tecnologías',
 '4,00',
 'A',
 'CLASE  ',
 'JUE',
 '11:30',
 '13:20',
 '30',
 'A-303',
 'CLASE  ',
 'MAR',
 '11:30',
 '13:20',
 '30',
 'X -302',
 '120253 - Antropología Filosófica',
 '4,00',
 'A']

In [22]:
course_sections = defaultdict(list)
course_credits = defaultdict(list)

current_course_code = None

# Split text by lines and process
for line in text.splitlines():
    # Match course code
    course_match = re.search(r"(\b\d{6}|1MN\d{3}|1F\d{4}\b)\s+-\s+", line)
    if course_match:
        current_course_code = course_match.group(1)
        # Check if the course code is in the list of interest
        if current_course_code not in list(df['course_code']):
            current_course_code = None  # Reset if not interested
        continue
    
    # Match section code if we have a current course of interest
    if current_course_code:
        section_match = re.search(r"\b^[A-Z]{1,2}$\b", line)
        credits_match = re.search(r"\b^(\d{1})\,\d{2}$\b", line)
        if section_match:
            section_code = section_match.group(0).strip()
            course_sections[current_course_code].append(section_code)
        if credits_match:
            credits = int(credits_match.group(1).strip())
            course_credits[current_course_code] = credits

for course in course_sections:
    course_sections[course] = sorted(list(set(course_sections[course])))
    

# Count sections for each course
course_section_counts = {course: len(sections) for course, sections in course_sections.items()}

In [23]:
course_section_size = defaultdict(list)

current_course_code = None
current_code_section = None

for line in text.splitlines():
    course_match = re.search(r"(\b\d{6}|1MN\d{3}|1F\d{4}\b)\s+-\s+", line)
    if course_match:
        current_course_code = course_match.group(1)
        # Check if the course code is in the list of interest
        if current_course_code not in list(df['course_code']):
            current_course_code = None  # Reset if not interested
        continue
    if current_course_code:
        section_match = re.search(r"\b^[A-Z]{1,2}$\b", line)
        if section_match:
            current_code_section = current_course_code + "-" + section_match.group(0).strip()
    if current_code_section and current_course_code:
        class_size_match = re.search(r'\b^\d{1,3}$\b', line)
        if class_size_match:
            class_size = class_size_match.group(0).strip()
            course_section_size[current_code_section] = int(class_size)

In [24]:
# Display results
for i, (course_section, size) in enumerate(course_section_size.items()):
    if i>=20:
        break
    print(f"{course_section} size: {size} students")

126177-A size: 30 students
120133-A size: 31 students
120133-AA size: 30 students
120133-AB size: 35 students
120133-AC size: 30 students
120133-B size: 30 students
120133-C size: 30 students
120133-D size: 35 students
120133-E size: 35 students
120133-F size: 30 students
120133-G size: 30 students
120133-H size: 30 students
120133-I size: 30 students
120133-J size: 30 students
120133-K size: 30 students
120133-L size: 30 students
120133-M size: 30 students
120133-N size: 30 students
120133-O size: 30 students
120133-P size: 30 students


In [25]:
class_size_df = pd.DataFrame(list(course_section_size.items()), columns=['course_code_section', 'size'])
class_size_df['course_code'] = class_size_df['course_code_section'].apply(lambda x: x[:6])
class_size_df.head(10)

,course_code_section,size,course_code
0,126177-A,30,126177
1,120133-A,31,120133
2,120133-AA,30,120133
3,120133-AB,35,120133
4,120133-AC,30,120133
5,120133-B,30,120133
6,120133-C,30,120133
7,120133-D,35,120133
8,120133-E,35,120133
9,120133-F,30,120133


In [26]:
course_size_df = class_size_df[['course_code', 'size']].groupby('course_code').sum().reset_index()
course_size_df.head(10)

,course_code,size
0,120000,7
1,120001,576
2,120006,510
3,120070,315
4,120125,450
5,120133,901
6,120199,27
7,122005,117
8,124019,135
9,124020,60


In [27]:
df = pd.merge(df, course_size_df, on='course_code', how='left')

In [28]:
# Display results
for i, (course, count) in enumerate(course_section_counts.items()):
    if i>=20:
        break
    print(f"Course {course}: {count} sections")

Course 126177: 1 sections
Course 120133: 29 sections
Course 129007: 13 sections
Course 122005: 4 sections
Course 120199: 1 sections
Course 120070: 10 sections
Course 124019: 4 sections
Course 120001: 21 sections
Course 120006: 17 sections
Course 124020: 2 sections
Course 127039: 1 sections
Course 120000: 12 sections
Course 120125: 15 sections
Course 130454: 1 sections
Course 130289: 1 sections
Course 130639: 1 sections
Course 130424: 4 sections
Course 138425: 3 sections
Course 132641: 21 sections
Course 130642: 14 sections


In [29]:
for i, (course_code, sections) in enumerate(course_sections.items()):
    if i>=20:
        break
    print(f"{course_code}: {sections}")

126177: ['A']
120133: ['A', 'AA', 'AB', 'AC', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
129007: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M']
122005: ['A', 'B', 'C', 'D']
120199: ['A']
120070: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
124019: ['A', 'B', 'C', 'D']
120001: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'V']
120006: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R']
124020: ['A', 'B']
127039: ['A']
120000: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']
120125: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']
130454: ['A']
130289: ['A']
130639: ['A']
130424: ['A', 'B', 'C', 'D']
138425: ['A', 'B', 'C']
132641: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U']
130642: ['A', 'C', 'D

In [30]:
for i, (course_code, credits) in enumerate(course_credits.items()):
    if i >= 20:
        break
    print(f"{course_code}: {credits}")

126177: 3
120133: 4
129007: 4
122005: 4
120199: 3
120070: 4
124019: 4
120001: 4
120006: 4
124020: 4
127039: 4
120000: 0
120125: 4
130454: 3
130289: 2
130639: 4
130424: 5
138425: 5
132641: 5
130642: 5


In [31]:
sections_df = pd.DataFrame(list(course_section_counts.items()), columns=['course_code', 'sections'])
credits_df = pd.DataFrame(list(course_credits.items()), columns=['course_code', 'credits'])

## Consolidating final Dataframe

### Merging aditional information

In [32]:
df = pd.merge(df, sections_df, on='course_code', how='left')
df = pd.merge(df, credits_df, on='course_code', how='left')
df['available_2024_2'] = df['sections'].notna()

In [33]:
df

,filepath,filename,text,group_project,teamwork,course_code,faculty,frshmn_administración,frshmn_contabilidad,frshmn_derecho,...,frshmn_ingeniería_empresarial,frshmn_ingeniería_en_innovacion_y_diseño,frshmn_marketing,frshmn_negocios_internacionales,frshmn_politica_filosofia_y_economia,frshmn_any,size,sections,credits,available_2024_2
0,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(1) 142085 - Gestión de la Sostenibilidad Soc...,Sílabo\n142085 - Gestión de la Sostenibilidad ...,False,True,142085,Administración,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,72.0,3.0,4.0,True
1,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(1) 144741 - Gestión de Personas (2023-I).pdf,Sílabo\n144741 - Gestión de Personas\nI.\nInfo...,False,False,144741,Administración,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,120.0,4.0,4.0,True
2,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(2) 142085 - Gestión de la Sostenibilidad Soc...,\n1/12 \n \n \n \nSÍLABO \n \nI. \nInformació...,False,True,142085,Administración,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,72.0,3.0,4.0,True
3,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(2) 144741 - Gestión de Personas (2023-I).pdf,\n \n \n1 \n \nSÍLABO \nI. Información g...,False,False,144741,Administración,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,120.0,4.0,4.0,True
4,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,140663 - Negocios Inclusivos (2023-I).pdf,Sílabo\n140663 - Negocios Inclusivos\nI.\nInfo...,False,False,140663,Administración,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,1MN384 - Plan de Marketing (2023-I).pdf,\n1/13 \n \n \n \nSÍLABO \n \nI. \nInformació...,False,False,1MN384,Marketing y Negocios,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,25.0,1.0,4.0,True
272,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,1MN385 - Pricing (2023-I).pdf,Sílabo\n1MN385 - Pricing\nI.\nInformación gene...,False,False,1MN385,Marketing y Negocios,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,3.0,True
273,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,1MN388 - Marketing de Contenidos (2023-I).pdf,\n1/3 \n \n \n \nSÍLABO \n \nI. \nInformación...,True,False,1MN388,Marketing y Negocios,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,False
274,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,1MN444 - Marketing Internacional en China y As...,SÍLABO \nI. \nInformación General \n \n• \nNo...,True,False,1MN444,Marketing y Negocios,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,30.0,1.0,3.0,True


### Filtering potential courses

If we only consider courses with group projects that are available on the second term of 2024, we find too many posible options

In [34]:
selection_0 = df[df['group_project'] & df['available_2024_2']].reset_index(drop=True)
print(f'There are {selection_0.shape[0]} potential courses without restrictions')

(63, 24)

Our ideal course for the upcoming term should meet the following criteria:

1. **Includes a group project**: This will facilitate peer interaction, which is crucial for our experimental design.
2. **Mandatory for freshmen**: Freshmen are less likely to have established social networks within the university. Additionally, freshmen are systematically assigned to their classrooms, which could be beneficial for our design.
3. **Offered by the Economics Department**: We have the full support of the economics faculty, unlike other departments where professors might resist giving up control over group formation or conducting surveys.
4. **Enrolled by a large number of students**: A large student pool is essential to ensure statistical power.

In [35]:
selection_1 = df[df['group_project'] & df['available_2024_2'] & df['frshmn_any'] & (df['faculty']=='Economía') & (df['size']>100)].reset_index(drop=True)
print(f'There are {selection_1.shape[0]} potential courses under our ideal restrictions')

There are 63 potential courses under our ideal restrictions


As there are no courses with such characteristics, we will have to evaluate a trade-off between some of them.

For instance, if we give up the restriction of it being from the Economics Department:

In [36]:
selection_2 = df[df['group_project'] & df['available_2024_2'] & df['frshmn_any'] & (df['size']>100)].reset_index(drop=True)
selection_2

,filepath,filename,text,group_project,teamwork,course_code,faculty,frshmn_administración,frshmn_contabilidad,frshmn_derecho,...,frshmn_ingeniería_empresarial,frshmn_ingeniería_en_innovacion_y_diseño,frshmn_marketing,frshmn_negocios_internacionales,frshmn_politica_filosofia_y_economia,frshmn_any,size,sections,credits,available_2024_2
0,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,141038 - Fundamentos de las Ciencias Empresari...,Sílabo\n141038 - Fundamentos de las Ciencias E...,True,False,141038,Administración,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,134.0,8.0,4.0,True
1,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,160092 - Fundamentos de Contabilidad (2023-I).pdf,Sílabo\n160092 - Fundamentos de Contabilidad\n...,True,False,160092,Contabilidad,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,507.0,19.0,4.0,True


And if we give up the restriction of it being mandatory for freshmen while keeping the Econ Department restriction:

In [37]:
selection_3 = df[df['group_project'] & df['available_2024_2'] & (df['faculty']=='Economía') & (df['size']>100)].reset_index(drop=True)
selection_3

,filepath,filename,text,group_project,teamwork,course_code,faculty,frshmn_administración,frshmn_contabilidad,frshmn_derecho,...,frshmn_ingeniería_empresarial,frshmn_ingeniería_en_innovacion_y_diseño,frshmn_marketing,frshmn_negocios_internacionales,frshmn_politica_filosofia_y_economia,frshmn_any,size,sections,credits,available_2024_2
0,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,(2) 138201 - Microeconomía I (2023-I).pdf,\n \n \n \n \n \nPROGRAMA DEL CURSO \n \n \nI...,True,True,138201,Economía,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,204.0,7.0,5.0,True
1,c:\Users\Matias Villalba\Documents\GitHub\VRI_...,134427 - Microeconomía II (2023-I).pdf,\n \n \n \n \n \nPROGRAMA DEL CURSO \n \n \nI...,True,False,134427,Economía,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,170.0,5.0,5.0,True


In [38]:
# Create clickable links to the PDF files
for df in [selection_0, selection_2, selection_3]:
    df['local_link'] = df['filepath'].apply(lambda x: f'<a href="{x}" target="_blank">pdf</a>')

In [39]:
# Function to keep only the first three levels of parent directories
def keep_three_levels(filepath):
    # Split the path by backslashes
    parts = filepath.split('\\')
    # Join the first four parts (three levels + file name)
    return '\\'.join(parts[-4:])

# Apply the function to the 'filepaths' column
for df in [selection_0, selection_2, selection_3]:
    df['online_link'] = 'https://mvillalbao.github.io/assets/pdf/VRI_UP_Networks_data/' + df['filepath'].apply(lambda x: keep_three_levels(x).replace('\\', '/'))
    df['online_link'] = df['online_link'].apply(lambda x: f'<a href="{x}" target="_blank">pdf</a>')

## Visualizing selected options

In [40]:
titles = ['Group project & For freshmen & Large pool of students', 'Group project & From Economics Department & Large pool of students']
dataframes = [selection_2, selection_3]

for title, df in zip(titles, dataframes):
    display(HTML(f'<h3 style="text-align:center;"><b>{title}</b></h3>'))
    display(HTML(df[['filename', 'size', 'sections', 'faculty', 'local_link', 'online_link']].to_html(escape=False, index=False)))

filename,size,sections,faculty,local_link,online_link
141038 - Fundamentos de las Ciencias Empresariales (2023-I).pdf,134.0,8.0,Administración,pdf,pdf
160092 - Fundamentos de Contabilidad (2023-I).pdf,507.0,19.0,Contabilidad,pdf,pdf


filename,size,sections,faculty,local_link,online_link
(2) 138201 - Microeconomía I (2023-I).pdf,204.0,7.0,Economía,pdf,pdf
134427 - Microeconomía II (2023-I).pdf,170.0,5.0,Economía,pdf,pdf
